#Laboratorio 4: Predicción y Decisión de umbral
#Analytics y Big Data: Ciencia de los Datos aplicada al mundo de los negocios

En este laboratorio, aplicaremos el mejor modelo encontrado con los datos de crédito (SVM) sobre nuevos clientes que no sabemos su comportamiento. De esta forma emulamos lo que sería una eventual aplicación de un modelo predictivo.

Primero importamos las librerías a utilizar:

In [35]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import pandas.util.testing as tm
from joblib import dump, load

Volvemos a montar google drive para que reconozca nuestra base de datos:

**Cuando corra la siguiente linea, le pedira un código, por favor siga las intrucciones a continuación, inserte el código entregado y presione enter**

Cargamos el archivo con la base de datos de testeo (segunda hoja del excel):

In [36]:
path = "C:\\Users\\PC SITOS\\AnalyticsYbigData\\creditSIM.xlsx"
data_set = pd.read_excel(path, sheet_name = 'prediccion')

Cargamos ahora el archivo con el modelo entrenado:

In [37]:
modelo_entrenado = load('C:\\Users\\PC SITOS\\AnalyticsYbigData\\modelo_entrenado.joblib')

Nuestro objeto data_set tiene las variables independientes que nos ayudarán a predecir la variable objetivo (si cliente cae en default o no). 

Antes de aplicar el modelo, debemos replicar el preprocesamiento a este nuevo *set* de datos.

#Preprocesamiento de datos de predicción

Veamos que contiene nuestra base de datos de predicción:


In [38]:
print(data_set)

         ID  AgnosDirec  AgnosEmpleo      DeudaExt     DeudaInt  Edad  \
0    103001         9.0          5.0   5831.594611  2509.745975    23   
1    103002         4.0          4.0   1305.171417    82.209945    41   
2    103003         8.0          4.0  11519.216306  3876.960332    39   
3    103004         1.0          2.0    854.973405   927.250911    53   
4    103005        11.0          7.0   6910.126741  1369.823025    53   
..      ...         ...          ...           ...          ...   ...   
995  103996         7.0          6.0   1202.501905   206.036968    47   
996  103997         4.0          5.0   1905.313132  2042.001078    34   
997  103998        13.0          3.0   6995.085180  1365.744698    21   
998  103999         6.0          3.0    873.790012   662.237955    46   
999  104000         0.0          0.0     47.518211    25.035199    35   

          Ingreso      Ingreso2 Nacionalidad NivelEdu  
0    78209.280104  61359.560021            N   SupCom  
1    25328.

In [39]:
data_set.describe(include='all')

,ID,AgnosDirec,AgnosEmpleo,DeudaExt,DeudaInt,Edad,Ingreso,Ingreso2,Nacionalidad,NivelEdu
count,1000.000000,999.000000,998.000000,999.000000,998.000000,1000.000000,999.000000,1000.000000,1000,1000
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,5
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,SupInc
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000,334
mean,103500.500000,7.355355,6.913828,3914.243122,2065.436561,36.789000,58707.868428,49285.833051,NaN,NaN
std,288.819436,7.325584,6.545925,5529.459045,4547.314237,9.808297,46424.137015,39255.939446,NaN,NaN
min,103001.000000,0.000000,0.000000,41.538809,5.553404,18.000000,6829.598681,0.000000,NaN,NaN
25%,103250.750000,2.000000,2.000000,947.274416,381.562707,29.750000,28656.648227,23795.774298,NaN,NaN
50%,103500.500000,5.000000,5.000000,2133.930304,900.497434,36.000000,46762.745843,39010.929379,NaN,NaN
75%,103750.250000,10.000000,10.000000,4575.723755,2137.803214,44.000000,75077.549495,61567.786032,NaN,NaN


La estadística descriptiva indica que tenemos un ingreso cero (inconsistencia) y algunos valores perdidos:

In [40]:
data_set.loc[data_set['Ingreso2'] < 1, ['Ingreso2']] = np.nan

Remplazamos los valores perdidos por la media:

In [41]:
data_set['AgnosEmpleo'].fillna(data_set['AgnosEmpleo'].median(skipna =True), inplace=True)
data_set['AgnosDirec'].fillna(data_set['AgnosDirec'].median(skipna =True), inplace=True)
data_set['Ingreso'].fillna(data_set['Ingreso'].median(skipna =True), inplace=True)
data_set['Ingreso2'].fillna(data_set['Ingreso2'].median(skipna =True), inplace=True)
data_set['DeudaInt'].fillna(data_set['DeudaInt'].median(skipna =True), inplace=True)
data_set['DeudaExt'].fillna(data_set['DeudaExt'].median(skipna =True), inplace=True)

Creamos la variable de endeudamiento (ratio deuda total sobre ingreso)


In [42]:
data_set['DeudaIng'] = (data_set['DeudaInt'] + data_set['DeudaExt'])/data_set['Ingreso2']

Ahora que no existen inconsistencias en nuestra base de datos, transformamos las variables financieras a su logaritmos (debemos hacer EXACTAMENTE lo mismo que en el modelo original).

In [43]:
data_set['Ln_Ingreso'] = np.log(data_set['Ingreso']+1)
data_set['Ln_Ingreso2'] = np.log(data_set['Ingreso2']+1)
data_set['Ln_DeudaExt'] = np.log(data_set['DeudaExt']+1)
data_set['Ln_DeudaInt'] = np.log(data_set['DeudaInt']+1)
del data_set['Ingreso']
del data_set['Ingreso2']
del data_set['DeudaExt']
del data_set['DeudaInt']

Creamos ahora un nuevo dataframe, X_pre, que tendrá las 7 variables relavantes que incluía el modelo de SVM.

In [44]:
X_pre = pd.DataFrame(data_set[['AgnosDirec','AgnosEmpleo','Edad','DeudaIng','Ln_Ingreso2','Ln_DeudaExt','Ln_DeudaInt']])

In [45]:
X_pre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   AgnosDirec   1000 non-null   float64
 1   AgnosEmpleo  1000 non-null   float64
 2   Edad         1000 non-null   int64  
 3   DeudaIng     1000 non-null   float64
 4   Ln_Ingreso2  1000 non-null   float64
 5   Ln_DeudaExt  1000 non-null   float64
 6   Ln_DeudaInt  1000 non-null   float64
dtypes: float64(6), int64(1)
memory usage: 54.8 KB


Normalizamos la base de datos:

In [46]:
scaler = MinMaxScaler()
scaler_data = scaler.fit(X_pre)
X = pd.DataFrame(scaler_data.transform(X_pre), index= X_pre.index, columns= X_pre.columns)

#Predicción

Con el modelo de entrenamiento y las variables independientes ya definidas y procesadas, podemos realizar la predicción de la variable objetivo como se muestra a continuación:

In [47]:
prediccion = modelo_entrenado.predict(X)

Podemos ver los resultados de predicción si imprimimos el objeto "prediccion", que toma el valor 1 cuando el cliente se predice como mal pagador ("default" o "s") y 0 en caso contrario ("no default" o "n").

In [48]:
print(prediccion)

[0 0 1 0 1 1 0 0 1 0 0 1 1 1 0 1 1 0 1 1 0 0 1 1 0 1 1 1 0 0 0 1 1 0 0 1 0
 1 1 0 0 1 0 0 0 1 1 1 1 1 0 0 0 0 0 1 1 0 1 0 0 1 0 1 0 0 0 0 0 1 1 0 0 0
 1 1 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 1 1 0 0 1 1 0 0 1
 1 0 1 1 0 0 0 1 0 1 0 0 1 0 1 0 1 1 1 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 1 1 0
 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1 0 1 0 0 1 1 1 1 1 1 0 1 0 0 0 0 1 0 0 1 0 0
 0 0 0 1 0 1 0 1 0 1 0 0 1 0 0 0 0 0 1 0 0 1 1 0 1 1 0 1 1 1 0 0 0 0 0 1 0
 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 1 0 1
 0 0 1 0 1 1 0 0 1 0 1 0 0 0 0 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 1 0 1 1
 0 0 1 0 1 1 1 0 0 0 1 1 0 0 1 0 1 1 1 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 1 0 1
 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0
 0 0 0 1 0 1 1 1 0 1 0 1 0 1 0 0 1 0 1 0 1 1 0 0 1 1 0 0 0 0 0 1 1 1 0 1 0
 0 1 0 0 1 0 1 1 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 1 0 0 0 1 1 0 0 0 1 0
 0 1 1 0 1 1 0 1 1 0 1 1 0 0 1 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 1 1 0 0 0 1 0
 1 1 0 0 0 0 1 1 0 0 1 1 

Para realizar esta predicción, el modelo utiliza un umbral de 0.5 para la probabilidad de incumplimiento. Esto quiere decir que si p>0.5 "prediccion" toma el valor de 1, y cuando sea menor a este umbral toma el valor de 0.

Pero cual es la mejor decisión de umbral? Contamos la cantidad de clientes predichos que caen en default:

In [49]:
prediccion.sum()/prediccion.size

0.393

Recordemos que la tasa de clientes malos es de un 20% en la base original. Nuestro modelo predice, sin embargo, más de un 38% de clientes malos pagadores. Esto se debe a la corrección que hace SMOTE, que le hace creer al modelo que hay la misma cantidad tanto de clientes buenos como malos.

Podemos modificar este umbral según lo que necesitamos, definiendo por ejemplo el umbral de 0.6 para tener una política de otorgamiento menos estricta. Una política muy restrictiva puede hacernos perder participación de mercado. 

In [50]:
prediccion_2 = (modelo_entrenado.predict_proba(X)[:,1] >= 0.6).astype(int)

In [51]:
print(prediccion_2)

[0 0 1 0 1 1 0 0 1 0 0 1 1 1 0 1 1 0 0 1 0 0 1 1 0 1 1 0 0 0 0 1 1 0 0 1 0
 1 1 0 0 1 0 0 0 1 1 1 0 1 0 0 0 0 0 1 1 0 1 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0
 1 1 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 0 1 1 0 0 1
 1 0 1 0 0 0 0 1 0 1 0 0 1 0 1 0 1 1 1 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 1 1 0
 0 1 1 0 0 1 1 0 0 0 1 0 0 1 1 0 1 0 0 1 0 1 1 1 0 0 1 0 0 0 0 1 0 0 1 0 0
 0 0 0 1 0 1 0 1 0 1 0 0 1 0 0 0 0 0 1 0 0 1 1 0 1 1 0 1 1 0 0 0 0 0 0 1 0
 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 1 0 1
 0 0 1 0 1 1 0 0 1 0 1 0 0 0 0 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 1
 0 0 1 0 1 1 1 0 0 0 1 1 0 0 1 0 1 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 1 0 1
 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0
 0 0 0 1 0 1 1 1 0 1 0 1 0 1 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 1 0 1 0
 0 0 0 0 1 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 1 1 0 0 0 1 0
 0 1 1 0 0 0 0 1 1 0 1 0 0 0 1 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 1 1 0 0 0 1 0
 1 1 0 0 0 0 1 1 0 0 1 1 

Nuevamente contamos la cantidad de defaulters con este nuevo umbral:

In [52]:
prediccion_2.sum()/prediccion_2.size

0.35

Podemos ver que los *defaulters* disminuyen al aumentar el umbral (desde un 38% a un 33% aproximadamente). Es posible modificar este criterio según los costos y beneficios del modelo.

#Exportar datos a excel

Finalmente, armamos la base de datos con todas las variables originales y le agregamos la variable predicha para tener toda la información del cliente y luego exportar a excel estos datos y poder trabajar con ellos.

Primero tenemos que transformar las variables que transformamos a logaritmos a su escala original

In [53]:
data_set['Ingreso'] = np.exp(data_set['Ln_Ingreso'])-1
data_set['Ingreso2'] = np.exp(data_set['Ln_Ingreso2'])-1
data_set['DeudaExt'] = np.exp(data_set['Ln_DeudaExt'])-1
data_set['DeudaInt'] = np.exp(data_set['Ln_DeudaInt'])-1
del data_set['Ln_Ingreso']
del data_set['Ln_Ingreso2']
del data_set['Ln_DeudaExt']
del data_set['Ln_DeudaInt']

Ahora con las variables listas agregamos la variable de predicción a nuestra base de datos

In [54]:
data_set['VarObj'] = prediccion_2

Por último, exportamos la base de datos a excel y la guardamos en nuestro drive:

In [55]:
path = "C:\\Users\\PC SITOS\\AnalyticsYbigData\\creditSIM_prediccion.csv"
data_set.to_csv(path, index=False)